In [1]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 80658, done.
remote: Counting objects: 100% (195/195), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 80658 (delta 118), reused 169 (delta 99), pack-reused 80463
Receiving objects: 100% (80658/80658), 594.59 MiB | 26.89 MiB/s, done.
Resolving deltas: 100% (57471/57471), done.


In [2]:
!cd models/research/ && protoc object_detection/protos/*.proto --python_out=.

In [3]:
!cd models/research && \
     cp object_detection/packages/tf2/setup.py . \
     && python -m pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 KB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 KB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [21]:
import os
import pathlib
import tensorflow as tf
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import imageio

In [5]:
gpus=tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu,True)

In [6]:
!wget -nc https://lazyprogrammer.me/cnn_class2_videos.zip

--2023-01-22 21:53:22--  https://lazyprogrammer.me/cnn_class2_videos.zip
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210, 2606:4700:3030::ac43:d5a6, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2073140 (2.0M) [application/zip]
Saving to: ‘cnn_class2_videos.zip’

cnn_class2_videos.z 100%[===================>]   1.98M  --.-KB/s    in 0.07s   

2023-01-22 21:53:23 (29.0 MB/s) - ‘cnn_class2_videos.zip’ saved [2073140/2073140]



In [7]:
!unzip cnn_class2_videos.zip

Archive:  cnn_class2_videos.zip
  inflating: catdog.mp4              
  inflating: safari.mp4              
  inflating: traffic.mp4             


In [26]:
INPUT_VIDEOS=['catdog','safari','traffic']

In [9]:
URL_PATH='http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz'

In [10]:
PATH_TO_MODEL_DIR=tf.keras.utils.get_file(fname='ssd_resnet101_v1_fpn_640x640_coco17_tpu-8',
                                          origin=URL_PATH,
                                          untar=True)

386527459/386527459 [==============================] - 2s 0us/step


In [11]:
PATH_TO_MODEL_DIR

'/root/.keras/datasets/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8'

In [12]:
LABEL_URL='https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt'

In [13]:
PATH_TO_LABELS=tf.keras.utils.get_file(fname='mscoco_label_map.pbtxt',
                                          origin=LABEL_URL,
                                          untar=False,extract=True)

5056/5056 [==============================] - 0s 0us/step


In [14]:
PATH_TO_LABELS

'/root/.keras/datasets/mscoco_label_map.pbtxt'

In [15]:
!head mscoco_label_map.pbtxt.txt

item {
  name: "/m/01g317"
  id: 1
  display_name: "person"
}
item {
  name: "/m/0199g"
  id: 2
  display_name: "bicycle"
}


In [16]:
PATH_TO_SAVED_MODEL=PATH_TO_MODEL_DIR+"/saved_model"

In [17]:
detect_fn=tf.saved_model.load(PATH_TO_SAVED_MODEL)

In [18]:
 category_index=label_map_util.create_category_index_from_labelmap('mscoco_label_map.pbtxt.txt', use_display_name=True)

In [19]:
category_index

{1: {'id': 1, 'name': 'person'},
 2: {'id': 2, 'name': 'bicycle'},
 3: {'id': 3, 'name': 'car'},
 4: {'id': 4, 'name': 'motorcycle'},
 5: {'id': 5, 'name': 'airplane'},
 6: {'id': 6, 'name': 'bus'},
 7: {'id': 7, 'name': 'train'},
 8: {'id': 8, 'name': 'truck'},
 9: {'id': 9, 'name': 'boat'},
 10: {'id': 10, 'name': 'traffic light'},
 11: {'id': 11, 'name': 'fire hydrant'},
 13: {'id': 13, 'name': 'stop sign'},
 14: {'id': 14, 'name': 'parking meter'},
 15: {'id': 15, 'name': 'bench'},
 16: {'id': 16, 'name': 'bird'},
 17: {'id': 17, 'name': 'cat'},
 18: {'id': 18, 'name': 'dog'},
 19: {'id': 19, 'name': 'horse'},
 20: {'id': 20, 'name': 'sheep'},
 21: {'id': 21, 'name': 'cow'},
 22: {'id': 22, 'name': 'elephant'},
 23: {'id': 23, 'name': 'bear'},
 24: {'id': 24, 'name': 'zebra'},
 25: {'id': 25, 'name': 'giraffe'},
 27: {'id': 27, 'name': 'backpack'},
 28: {'id': 28, 'name': 'umbrella'},
 31: {'id': 31, 'name': 'handbag'},
 32: {'id': 32, 'name': 'tie'},
 33: {'id': 33, 'name': 'suitc

In [32]:
def detect_objects(image_np):
  input_tensor=tf.convert_to_tensor(image_np)
  input_tensor=input_tensor[tf.newaxis, ...]
  detections=detect_fn(input_tensor)
  num_detections=int(detections.pop('num_detections')) 
  detections={key: value[0,:num_detections].numpy()
                for key,value in detections.items()}
  detections['num_detections']=num_detections
  detections['detection_classes']=detections['detection_classes'].astype(np.int64)
  image_np_with_detections=image_np.copy()

  viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes'],
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=200,
            min_score_thresh=.5,
            agnostic_mode=False)
  return image_np_with_detections

In [33]:
def detect_object_in_video(input_video):
  video_reader=imageio.get_reader(f'{input_video}.mp4')
  video_writer=imageio.get_writer(f'{input_video}_annotated.mp4',fps=10)
  n_frames=0
  for frame in video_reader:
    n_frames+=1
    new_frame=detect_objects(frame)
    video_writer.append_data(new_frame)

  video_writer.close()




In [34]:
detect_object_in_video(INPUT_VIDEOS[0])

In [35]:
detect_object_in_video(INPUT_VIDEOS[1])

In [36]:
detect_object_in_video(INPUT_VIDEOS[2])